# transformations

> Classes and functions that deal with transforming text during notebook export

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp markdown.transformations

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
#| export
from contextlib import contextmanager
import io
import re
from textwrap import dedent
from typing import Callable, Dict, Iterable, Optional, Sequence, Union

In [ ]:
# | export


class Transformer:
    """Base class for all content transformers."""

    def emit_before(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            before the set of lines this transformer processes."""
        return

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        """Implement this method on sub-classes to modify the lines \
            this transformer processes."""
        return lines

    def emit_after(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            after the set of lines this transformer processes."""
        return

In [ ]:
show_doc(Transformer.emit_before)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L16){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_before

>      Transformer.emit_before (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             before the set of lines this transformer processes.

In [ ]:
show_doc(Transformer.process_lines)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L21){target="_blank" style="float:right; font-size:smaller"}

### Transformer.process_lines

>      Transformer.process_lines (lines:Sequence[str])

Implement this method on sub-classes to modify the lines             this transformer processes.

In [ ]:
show_doc(Transformer.emit_after)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L26){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_after

>      Transformer.emit_after (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             after the set of lines this transformer processes.

In [ ]:
# | export


class TransformerWithDirectives(Transformer):
    """Base class for transformers that use directives."""

    def __init__(self) -> None:
        self.directives: Dict[str, Optional[Union[bool, str]]] = {}
        super().__init__()

    @contextmanager
    def begin_using_directives(self, directives: Dict[str, Optional[Union[bool, str]]]):
        """
        Factory function for a context manager that ensures the given\
         directives are cleared after they are used.

        Parameters
        ----------
        directives: A map of directives to be used in this transformer's
                    processing

        Returns
        -------
        A context manager that will clear the directives after the context
        has exited.
        """
        self.directives = directives
        yield
        self.directives = {}

In [ ]:
show_doc(TransformerWithDirectives.begin_using_directives)

---

### TransformerWithDirectives.begin_using_directives

>      TransformerWithDirectives.begin_using_directives
>                                                        (directives:Dict[str,Op
>                                                        tional[bool]])

Factory function for a context manager that ensures the given         directives are cleared after they are used.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| directives | Dict |  |
| **Returns** | **A context manager that will clear the directives after the context** |  |

In [ ]:
# | export
class MultiTransformer(Transformer):
    def __init__(self, transformers: Iterable[Transformer]):
        # Store the passed in transformers as a list
        # so that we can later call reverse() on it.
        self.transformers = list(transformers)

    def emit_before(self, stream: io.TextIOBase):
        # Emit all the before output from the transformers.
        # Do it in reversed order so that the first transformer's
        # before output appears closest to the (transformed) lines
        for transformer in reversed(self.transformers):
            transformer.emit_before(stream)

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        # Pass the source lines through all the transformers
        for transformer in self.transformers:
            lines = transformer.process_lines(lines)
        return lines

    def emit_after(self, stream: io.TextIOBase):
        # Emit the after output from the transformers
        for transformer in self.transformers:
            transformer.emit_after(stream)

In [ ]:
# | export

def emit_with_transformation(
    transformer: Transformer,
    lines: Sequence[str],
    emit_lines_func: Callable[[Sequence[str], io.TextIOBase], None],
    stream: io.TextIOBase,
):
    transformer.emit_before(stream)

    emit_lines_func(transformer.process_lines(lines), stream)

    transformer.emit_after(stream)


In [ ]:
# Tests for emit_with_transformations()
class TransformerA(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before A\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('a', 'A')
            for line in lines
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after A\n')

class TransformerB(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before B\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('b', 'B')
            for line in lines
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after B\n')

def dummy_emit_lines(lines: Sequence[str], stream: io.TextIOBase):
    for line in lines:
        stream.write(line)

lines = [
    'abCDefab\n',
    'ghiAbajkl\n'
    'mnoAAAAAABBBBBp\n'
]

stream = io.StringIO()

emit_with_transformation(
    transformer=MultiTransformer([TransformerA(), TransformerB()]),
    lines=lines,
    emit_lines_func=dummy_emit_lines,
    stream=stream
)

stream.seek(0)
output = stream.read()

expected = """\
before B
before A
ABCDefAB
ghiABAjkl
mnoAAAAAABBBBBp
after A
after B
"""

test_eq(output, expected)

In [ ]:
# | export
class ReplaceSingleDollarDelimiters(Transformer):
    """Transformer that replaces $ delimiters in inline latex\
        with \\\\( and \\\\)."""
    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex = r"(?<!\$)\$(?!\$)(.*?[^\\])\$(?!\$)(?!\w)"
        replacement = r"\\\\(\1\\\\)"

        return [re.sub(regex, replacement, line) for line in lines]

In [ ]:
# Test ReplaceSingleDollarDelimiters

input = [
    'This is an expression in which the delimiters will be replaced: $a + b$\n',
    'These block delimiters should not be replaced: $$a + b$$\n'
]

output = ReplaceSingleDollarDelimiters().process_lines(input)

expected = [
    r'This is an expression in which the delimiters will be replaced: \\(a + b\\)''\n',
    'These block delimiters should not be replaced: $$a + b$$\n'
]

test_eq(output, expected)


In [ ]:
# | export
class EscapeUnderscoresWithinLatexMath(Transformer):
    """Transformer that replaces underscores within latex\
        math expressions with \_."""

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex_inline = r"(?<!\$)\$(?!\$)(.*?[^\\])\$(?!\$)"
        regex_block = r"\$\$([\s\S]*?)\$\$"

        # We want to handle cases where math expressions could be in a single
        # line or spread across multiple lines. So we'll join the lines with
        # a dummy token separator into a single string, perform the substitutions
        # and the split back into lines on the dummy token.
        dummy_token = 'DUMMY+TOKEN+DO+NOT+USE'
        text = dummy_token.join(lines)

        # Escaping underscores in block math expressions
        text = re.sub(
            regex_block, lambda match: re.sub(r"_", r"\_", match.group()), text
        )

        # Escaping underscores in inline math expressions
        text = re.sub(
            regex_inline, lambda match: re.sub(r"_", r"\_", match.group()), text
        )

        processed_lines = text.split(dummy_token)
        return processed_lines

In [ ]:
# Test EscapeUnderscoresWithinLatexMath

test_cases = [
    {
        'name': 'Escape underscores',
        'lines': ['$a_1 + b_1$\n'],
        'expected': ['$a\_1 + b\_1$\n'],
    },
    {
        'name': 'Escape underscores',
        'lines': ['$$y_1 - x_1$$\n'],
        'expected': ['$$y\_1 - x\_1$$\n'],
    },
    {
        'name': 'Leave alone',
        'lines': ['$a + b$ and $$y - x$$\n'],
        'expected': ['$a + b$ and $$y - x$$\n'],
    },
    {
        'name': 'A block with some newlines',
        'lines': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
        'expected': [
            '$$\\begin{bmatrix}\n',
            '{x}\_{1,1} & {x}\_{1,2} & {x}\_{1,3} & {x}\_{1,4}\\\\\n',
            '{x}\_{2,1} & {x}\_{2,2} & {x}\_{2,3} & {x}\_{2,4}\\\\\n',
            '{x}\_{3,1} & {x}\_{3,2} & {x}\_{3,3} & {x}\_{3,4}\\\\\n',
            '{x}\_{4,1} & {x}\_{4,2} & {x}\_{4,3} & {x}\_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
    },
    {
        'name': 'Expressions both in single lines and across lines',
        'lines': [
            '$a_1 + b_1$\n',
            '$$y_1 - x_1$$\n' '\n',
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
        'expected': [
            '$a\_1 + b\_1$\n',
            '$$y\_1 - x\_1$$\n' '\n',
            '$$\\begin{bmatrix}\n',
            '{x}\_{1,1} & {x}\_{1,2} & {x}\_{1,3} & {x}\_{1,4}\\\\\n',
            '{x}\_{2,1} & {x}\_{2,2} & {x}\_{2,3} & {x}\_{2,4}\\\\\n',
            '{x}\_{3,1} & {x}\_{3,2} & {x}\_{3,3} & {x}\_{3,4}\\\\\n',
            '{x}\_{4,1} & {x}\_{4,2} & {x}\_{4,3} & {x}\_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
    },
    {
        'name': 'Pathological case: unterminated multi-line block (should not change)',
        'lines': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
        ],
        'expected': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
        ],
    },
]

for tc in test_cases:
    name = tc['name']
    lines = tc['lines']
    expected = tc['expected']

    print(f"Case: {name}")
    output = EscapeUnderscoresWithinLatexMath().process_lines(lines)
    test_eq(output, expected)

Case: Escape underscores
Case: Escape underscores
Case: Leave alone
Case: A block with some newlines
Case: Expressions both in single lines and across lines
Case: Pathological case: unterminated multi-line block (should not change)


In [ ]:
# | export
class EscapeEndLineSlashesWithinLatexMath(Transformer):
    """Transformer that replaces slashes ("\\") at the end of lines\
        within math block expressions expressions with "\\\\".\
        Handles the case where the "\\" is followed by a line width\
        e.g. "\\[2em]"."""

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex_block = r"\$\$([\s\S]*?)\$\$"

        # We want to handle cases where math expressions could be in a single
        # line or spread across multiple lines. So we'll join the lines with
        # a dummy token separator into a single string, perform the substitutions
        # and the split back into lines on the dummy token.
        dummy_token = 'DUMMY+TOKEN+DO+NOT+USE'
        text = dummy_token.join(lines)

        text = re.sub(
            regex_block,
            lambda match: re.sub(
                r"\\\\(?=\[.*?\]$|$)", r"\\\\\\\\", match.group(), flags=re.MULTILINE
            ),
            text,
        )

        processed_lines = text.split(dummy_token)
        return processed_lines

In [ ]:
# Test EscapeEndLineSlashesWithinLatexMath

test_cases = [
    {
        'name': 'A block across multiple lines',
        'lines': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
        'expected': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
    },
    {
        'name': 'A block within a line',
        'lines': ['A block within a line: $$\n\\begin{bmatrix}\nx_{1,1}, x_{1, 2}\\\\\n\\end{bmatrix}\n$$'],
        'expected': ['A block within a line: $$\n\\begin{bmatrix}\nx_{1,1}, x_{1, 2}\\\\\\\\\n\\end{bmatrix}\n$$'],
    },
    {
        'name': r'Line spacing specified at end of line e.g. \\[2em]',
        'lines': [
            '$$\n',
            '\\begin{align}\n',
            '\\frac{\\partial \\mathcal{L}}{\\partial W^{[2]}_{1, 1}} &= \\sum_{k, l} \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{k, l}}\\frac{\\partial z^{[2]}_{k, l}}{\\partial W^{[2]}_{1, 1}}\\\\[2em]\n',
            '&= \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{1, 1}}\\frac{\\partial z^{[2]}_{1, 1}}{\\partial W^{[2]}_{1, 1}} + \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{12}}\\frac{\\partial z^{[2]}_{12}}{\\partial W^{[2]}_{1, 1}} + \\cdots + \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{44}}\\frac{\\partial z^{[2]}_{44}}{\\partial W^{[2]}_{1, 1}} \n',
            '\\end{align}\n',
            '$$\n',
        ],
        'expected': [
            '$$\n',
            '\\begin{align}\n',
            '\\frac{\\partial \\mathcal{L}}{\\partial W^{[2]}_{1, 1}} &= \\sum_{k, l} \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{k, l}}\\frac{\\partial z^{[2]}_{k, l}}{\\partial W^{[2]}_{1, 1}}\\\\\\\\[2em]\n',
            '&= \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{1, 1}}\\frac{\\partial z^{[2]}_{1, 1}}{\\partial W^{[2]}_{1, 1}} + \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{12}}\\frac{\\partial z^{[2]}_{12}}{\\partial W^{[2]}_{1, 1}} + \\cdots + \\frac{\\partial \\mathcal{L}}{\\partial z^{[2]}_{44}}\\frac{\\partial z^{[2]}_{44}}{\\partial W^{[2]}_{1, 1}} \n',
            '\\end{align}\n',
            '$$\n',
        ]

    }
]

for tc in test_cases:
    name = tc['name']
    lines = tc['lines']
    expected = tc['expected']

    print(f"Case: {name}")
    output = EscapeEndLineSlashesWithinLatexMath().process_lines(lines)
    test_eq(output, expected)

Case: A block across multiple lines
Case: A block within a line
Case: Line spacing specified at end of line e.g. \\[2em]


In [ ]:
# | export
class EscapeEqualsSignsAtLineStartWithinLatexMath(Transformer):
    """Transformer that replaces equals signs ("=") at the start of lines\
        within math block expressions with "\=".\
        This is needed because lines that start with = are sometimes\
        interpreted as headings by markdown."""

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex_block = r"\$\$([\s\S]*?)\$\$"

        # We want to handle cases where math expressions could be in a single
        # line or spread across multiple lines. So we'll join the lines with
        # a dummy token separator into a single string, perform the substitutions
        # and the split back into lines on the dummy token.
        dummy_token = 'DUMMY+TOKEN+DO+NOT+USE'
        text = dummy_token.join(lines)

        text = re.sub(
            regex_block,
            lambda match: re.sub(
                rf"^{re.escape(dummy_token)}=", # dummy_token will be at the start of lines
                rf"{dummy_token}\\=",
                match.group(),
                flags=re.MULTILINE,
            ),
            text,
        )

        processed_lines = text.split(dummy_token)
        return processed_lines

In [ ]:
# Test EscapeEqualsSignsAtLineStartWithinLatexMath

test_cases = [
    {
        'name': 'Basic',
        'lines': [
            '$$\n',
            '=\n',
            '$$\n',
        ],
        'expected': [
            '$$\n',
            '\=\n',
            '$$\n',
        ],
    },
    {
        'name': 'Multiple equals',
        'lines': [
            '$$\n',
            '==\n',
            '$$\n',
        ],
        'expected': [
            '$$\n',
            '\==\n',
            '$$\n',
        ],
    },
    {
        'name': 'Real example',
        'lines': [
            '$$\n',
            '\\mathbf{B^{[2]}} = \n',
            '\\def\\arraystretch{1.5}\n',
            '\\begin{bmatrix}\n',
            '\\mathbf{B^{[2]}_{1, 1}} & \\mathbf{B^{[2]}_{1, 2}} & \\mathbf{B^{[2]}_{1, 3}} & \\mathbf{B^{[2]}_{1, 4}}\\\\\n',
            '\\mathbf{B^{[2]}_{2, 1}} & \\mathbf{B^{[2]}_{2, 2}} & \\mathbf{B^{[2]}_{2, 3}} & \\mathbf{B^{[2]}_{2, 4}}\\\\\n',
            '\\mathbf{B^{[2]}_{3, 1}} & \\mathbf{B^{[2]}_{3, 2}} & \\mathbf{B^{[2]}_{3, 3}} & \\mathbf{B^{[2]}_{3, 4}}\\\\\n',
            '\\mathbf{B^{[2]}_{4, 1}} & \\mathbf{B^{[2]}_{4, 2}} & \\mathbf{B^{[2]}_{4, 3}} & \\mathbf{B^{[2]}_{4, 4}}\\\\\n',
            '\\end{bmatrix}\n',
            '=\n',
            '\\begin{bmatrix}\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            '\\end{bmatrix}\n',
            '$$\n',
        ],
        'expected': [
            '$$\n',
            '\\mathbf{B^{[2]}} = \n',
            '\\def\\arraystretch{1.5}\n',
            '\\begin{bmatrix}\n',
            '\\mathbf{B^{[2]}_{1, 1}} & \\mathbf{B^{[2]}_{1, 2}} & \\mathbf{B^{[2]}_{1, 3}} & \\mathbf{B^{[2]}_{1, 4}}\\\\\n',
            '\\mathbf{B^{[2]}_{2, 1}} & \\mathbf{B^{[2]}_{2, 2}} & \\mathbf{B^{[2]}_{2, 3}} & \\mathbf{B^{[2]}_{2, 4}}\\\\\n',
            '\\mathbf{B^{[2]}_{3, 1}} & \\mathbf{B^{[2]}_{3, 2}} & \\mathbf{B^{[2]}_{3, 3}} & \\mathbf{B^{[2]}_{3, 4}}\\\\\n',
            '\\mathbf{B^{[2]}_{4, 1}} & \\mathbf{B^{[2]}_{4, 2}} & \\mathbf{B^{[2]}_{4, 3}} & \\mathbf{B^{[2]}_{4, 4}}\\\\\n',
            '\\end{bmatrix}\n',
            '\\=\n',
            '\\begin{bmatrix}\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            'b^{[2]} & b^{[2]} & b^{[2]} & b^{[2]}\\\\\n',
            '\\end{bmatrix}\n',
            '$$\n',
        ],
    },
    {
        'name': 'Not within latex math should get left alone',
        'lines': [
            'abc',
            '=\n',
            'def',
        ],
        'expected': [
            'abc',
            '=\n',
            'def',
        ],
    },
    {
        'name': 'Equals not at line start should get left alone',
        'lines': [
            '$$\n',
            'x = 2 - y\n',
            '$$\n',
        ],
        'expected': [
            '$$\n',
            'x = 2 - y\n',
            '$$\n',
        ],
    },
]

for tc in test_cases:
    name = tc['name']
    lines = tc['lines']
    expected = tc['expected']

    print(f"Case: {name}")
    output = EscapeEqualsSignsAtLineStartWithinLatexMath().process_lines(lines)
    test_eq(output, expected)

Case: Basic
Case: Multiple equals
Case: Real example
Case: Not within latex math should get left alone
Case: Equals not at line start should get left alone


In [ ]:
# | export
class Unindent(Transformer):
    """Transformer that removes leading indentation from a set\
        of lines. Will determine how far indented the first line \
        is and then remove \
        min(indentation of first non-empty line, indentation of current line) \
        for each subsequent line."""

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        i = 0
        for i, line in enumerate(lines):
            if line.strip() != '':
                break

        # i is now the index of the first line that is not all whitespace
        # or len(lines)-1 if all the lines were just whitespace.

        if i == len(lines) - 1 and line.strip() == '':
            # Everything was whitespace, just return it
            # unmodified
            return lines

        # If we're here, i is the index of the first line that is not
        # all whitespace.

        first_line_indent = len(lines[i]) - len(lines[i].lstrip())

        processed_lines = []
        for line in lines[i:]:
            cur_indent = len(line) - len(line.lstrip())
            remove_count = min(first_line_indent, cur_indent)
            processed_lines.append(line[remove_count:])

        return processed_lines




In [ ]:
# Test Unindent

test_cases = [
    {
        'name': 'basic',
        'lines': [
            '    line 2\n',
            '      - line 2\n',
            '      - line 3\n',
            '    line 4\n',
            '      - line 5\n',
            'line 6\n',
        ],
        'expected': [
            'line 2\n',
            '  - line 2\n',
            '  - line 3\n',
            'line 4\n',
            '  - line 5\n',
            'line 6\n',
        ]
    },
    {
        'name': 'all lines are whitespace - leave unmodified',
        'lines': [
            '     \n',
            '\n',
            '  \n',
        ],
        'expected': [
            '     \n',
            '\n',
            '  \n',
        ]
    },
    {
        'name': 'leading whitespace lines are stripped',
        'lines': [
            '     \n',
            '\n',
            '    line 2\n',
            '      - line 2\n',
            '      - line 3\n',
            '    line 4\n',
            '      - line 5\n',
            'line 6\n',
        ],
        'expected': [
            'line 2\n',
            '  - line 2\n',
            '  - line 3\n',
            'line 4\n',
            '  - line 5\n',
            'line 6\n',
        ]
    }

]

unindent = Unindent()

for tc in test_cases:
    name = tc['name']
    lines = tc['lines']
    expected = tc['expected']

    print(f"Case: {name}")
    output = unindent.process_lines(lines)
    test_eq(output, expected)


Case: basic
Case: all lines are whitespace - leave unmodified
Case: leading whitespace lines are stripped


In [ ]:
# | export
class CodeFoldTransformer(TransformerWithDirectives):
    """Transformer that implements code folding as per \
        [Quarto Code Output](https://quarto.org/docs/reference/cells/cells-jupyter.html#code-output)
        by emitting the Hugo `collapsible` shortcode."""
    def emit_before(self, stream: io.TextIOBase):
        code_fold = self.directives.get('code-fold', False)

        # code-fold must be explicitly set to True or the value 'show'
        # for code-folding to happen.
        if code_fold not in [True, 'show']:
            return

        summary = self.directives.get('code-summary', 'Code')

        # Sanity check there wasn't some crazy value for code-summary
        # in the directives.
        assert isinstance(summary, str)

        # Include the open param to the shortcode if code-fold was
        # set to "show".
        open_param = ''
        if code_fold == 'show':
            open_param = 'open=1 '

        stream.write(
            r'{{% collapsible class="code-fold" summary=' f'"{summary}" ' f'{open_param}' r'%}}''\n'
        )

    def emit_after(self, stream: io.TextIOBase):
        code_fold = self.directives.get('code-fold', False)
        # code-fold must be explicitly set to True or the value 'show'
        # for code-folding to happen.
        if code_fold not in [True, 'show']:
            return
        stream.write(r'{{% /collapsible %}}''\n')

In [ ]:
test_cases = [
    {
        'name': 'Code fold with no summary',
        'directives': {
            'code-fold': True,
        },
        'expected': dedent(
            """\
            {{% collapsible class="code-fold" summary="Code" %}}
            {{% /collapsible %}}
            """
        ),
    },
    {
        'name': 'Code fold with summary',
        'directives': {
            'code-fold': True,
            'code-summary': 'Here is some code'
        },
        'expected': dedent(
            """\
            {{% collapsible class="code-fold" summary="Here is some code" %}}
            {{% /collapsible %}}
            """
        ),
    },
    {
        'name': 'Code fold set to "show"',
        'directives': {
            'code-fold': 'show',
            'code-summary': 'Here is some code'
        },
        'expected': dedent(
            """\
            {{% collapsible class="code-fold" summary="Here is some code" open=1 %}}
            {{% /collapsible %}}
            """
        ),
    },
    {
        'name': 'Code fold explicitly false',
        'directives': {
            'code-fold': False,
        },
        'expected': '',
    },
    {
        'name': 'Code fold set to bogus value',
        'directives': {
            'code-fold': 'some bogus value',
        },
        'expected': '',
    },
    {
        'name': 'No code fold directive',
        'directives': {
            'some-other-directive': False,
            'yet-another-directive': True,
        },
        'expected': '',
    },
]

for tc in test_cases:
    name = tc['name']
    directives = tc['directives']
    expected = tc['expected']

    stream = io.StringIO()
    transformer = CodeFoldTransformer()

    with transformer.begin_using_directives(directives):
        transformer.emit_before(stream)
        transformer.emit_after(stream)

    print(f"Case: {name}")
    stream.seek(0)
    output = stream.read()

    test_eq(output, expected)

Case: Code fold with no summary
Case: Code fold with summary
Case: Code fold set to "show"
Case: Code fold explicitly false
Case: Code fold set to bogus value
Case: No code fold directive


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()